<a href="https://colab.research.google.com/github/v-gbarber/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 1s (158 kB/s)
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
# Read created csv file.
csv_file = "home_sales_revised.csv"

df=spark.read.csv(SparkFiles.get(csv_file), header=True, inferSchema=True)
df.show()


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [9]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query_avg_price ="""
SELECT YEAR(date) as year, ROUND (AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms=4
GROUP BY YEAR(date)
ORDER BY YEAR (date)
"""
results = spark.sql(query_avg_price)
results.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query_avg_price_built= """
SELECT YEAR(date) as year, ROUND (AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms=3 and bathrooms=3
GROUP BY YEAR(date)
ORDER BY YEAR (date)
"""
results_built = spark.sql(query_avg_price_built)
results_built.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [15]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price_built= """
SELECT YEAR(date) as year, ROUND (AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms=3 and bathrooms=3 and floors = 2 and sqft_living >=2000
GROUP BY YEAR(date)
ORDER BY YEAR (date)
"""
results_avg = spark.sql(avg_price_built)
results_avg.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
|2022|290242.99|
+----+---------+



In [16]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
view_rating= """
SELECT view, ROUND (AVG(price), 2) as avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view
"""
results_view = spark.sql(view_rating)
results_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 1.0602436065673828 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
view_rating= """
SELECT view, ROUND (AVG(price), 2) as avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view
"""
results_view = spark.sql(view_rating)
results_view.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 1.9118168354034424 seconds ---


In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
db_partitioned=("home_sales_parquet")
# Write a DataFrame parquet, partitionBy date_built
df.write.partitionBy("date_built").parquet(db_partitioned, mode="overwrite")

In [33]:
# 11. Read the parquet formatted data.
df_parquet= spark.read.parquet(db_partitioned)

In [35]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("homes_sales_parquet")

In [36]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
view_parquet_query= """
SELECT view, ROUND (AVG(price), 2) as avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view
"""
view_parquet_result = spark.sql(view_parquet_query)
view_parquet_result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.8670611381530762 seconds ---


In [37]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [39]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales") or spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
